# EDAするノートブック

In [1]:
!pip install l5kit
!pip install zarr

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


https://www.kaggle.com/kool777/lyft-level5-eda-training-inference

↑参考　これがなきゃマジで死んでた，感謝…！

In [10]:
from pathlib import Path
import pandas as pd
import numpy as np
import os
os.environ["L5KIT_DATA_FOLDER"] = '/home/knikaido/work/Lyft/data/lyft-motion-prediction-autonomous-vehicles'
import zarr
from prettytable import PrettyTable

from l5kit.data import PERCEPTION_LABELS
from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset

In [11]:
INPUT_ROOT = Path('/home/knikaido/work/Lyft/data/')
SAMPLE_ZARR = INPUT_ROOT / 'lyft-motion-prediction-autonomous-vehicles/scenes/sample.zarr'

cfg = load_config_data(str(INPUT_ROOT / "lyft-config-files/visualisation_config.yaml"))

In [12]:
sample_zarr = zarr.open(str(SAMPLE_ZARR))

In [13]:
sample_zarr.info

Name,/
Type,zarr.hierarchy.Group
Read-only,False
Store type,zarr.storage.DirectoryStore
No. members,5
No. arrays,4
No. groups,1
Arrays,"agents, frames, scenes, traffic_light_faces"
Groups,agents_mask


In [14]:
dm = LocalDataManager()
dataset_path = dm.require(cfg["val_data_loader"]["key"])
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()
print(zarr_dataset)

+------------+------------+------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|    100     |   24838    |  1893736   |       0.69      |        248.38        |        76.24         |        24.83         |        10.00        |
+------------+------------+------------+-----------------+----------------------+----------------------+----------------------+---------------------+


In [15]:
print(dataset_path)

/home/knikaido/work/Lyft/data/lyft-motion-prediction-autonomous-vehicles/scenes/sample.zarr


In [16]:
agents = pd.DataFrame.from_records(zarr_dataset.agents, columns = ['centroid', 'extent', 'yaw', 'velocity', 'track_id', 'label_probabilities'])
agents[['centroid_x','centroid_y']] = agents['centroid'].to_list()
agents = agents.drop('centroid', axis=1)
agents[['extent_x','extent_y', 'extent_z']] = agents['extent'].to_list()
agents = agents.drop('extent', axis=1)
agents[['velocity_x','velocity_y']] = agents['velocity'].to_list()
agents = agents.drop('velocity', axis=1)
agents.head()

,yaw,track_id,label_probabilities,centroid_x,centroid_y,extent_x,extent_y,extent_z,velocity_x,velocity_y
0,1.016675,1,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",665.034241,-2207.512207,4.391328,1.813830,1.590976,0.0,0.0
1,-0.783224,2,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",717.661255,-2173.760010,5.150925,1.953092,2.040210,0.0,0.0
2,-0.321747,3,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",730.681396,-2180.678955,2.948282,1.484217,1.112507,0.0,0.0
3,0.785501,4,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",671.253601,-2204.745361,1.706702,0.928787,0.628216,0.0,0.0
4,1.492359,5,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",669.776306,-2213.004639,0.251099,0.634378,1.654377,0.0,0.0


__やっっっっとzarrの中身見ることができた！！！！__

とりあえず，agentの中身を見れた．<br>
extentは，大きさ的なことみたい．

In [17]:
agents = zarr_dataset.agents
probabilities = agents["label_probabilities"]
labels_indexes = np.argmax(probabilities, axis=1)
counts = []
for idx_label, label in enumerate(PERCEPTION_LABELS):
    counts.append(np.sum(labels_indexes == idx_label))
    
table = PrettyTable(field_names=["label", "counts"])
for count, label in zip(counts, PERCEPTION_LABELS):
    table.add_row([label, count])
print(table)

+------------------------------------+---------+
|               label                |  counts |
+------------------------------------+---------+
|      PERCEPTION_LABEL_NOT_SET      |    0    |
|      PERCEPTION_LABEL_UNKNOWN      | 1324481 |
|     PERCEPTION_LABEL_DONTCARE      |    0    |
|        PERCEPTION_LABEL_CAR        |  519385 |
|        PERCEPTION_LABEL_VAN        |    0    |
|       PERCEPTION_LABEL_TRAM        |    0    |
|        PERCEPTION_LABEL_BUS        |    0    |
|       PERCEPTION_LABEL_TRUCK       |    0    |
| PERCEPTION_LABEL_EMERGENCY_VEHICLE |    0    |
|   PERCEPTION_LABEL_OTHER_VEHICLE   |    0    |
|      PERCEPTION_LABEL_BICYCLE      |    0    |
|    PERCEPTION_LABEL_MOTORCYCLE     |    0    |
|      PERCEPTION_LABEL_CYCLIST      |   6688  |
|   PERCEPTION_LABEL_MOTORCYCLIST    |    0    |
|    PERCEPTION_LABEL_PEDESTRIAN     |  43182  |
|      PERCEPTION_LABEL_ANIMAL       |    0    |
|     AVRESEARCH_LABEL_DONTCARE      |    0    |
+-------------------

__ラベルは，実際三種+不明しかない．不明のデータ数がめっっっっちゃおおい．__